In [1]:
from pathlib import Path
import pandas as pd
import sqlite3

import warnings
warnings.simplefilter(action='ignore')

In [2]:
in_dir = Path('/home/xn/Documents')
file = 'korona_hun - koronahun.xlsx'

df = pd.read_excel(in_dir/file)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 39 columns):
 #   Column                                                               Non-Null Count  Dtype         
---  ------                                                               --------------  -----         
 0   Dátum                                                                1030 non-null   datetime64[ns]
 1   Regisztrált esetek száma                                             723 non-null    float64       
 2   Új esetek száma                                                      723 non-null    float64       
 3   Elhunytak                                                            710 non-null    float64       
 4   Gyógyultak                                                           713 non-null    float64       
 5   Mintavétel száma                                                     723 non-null    float64       
 6   Egymillió lakosra elvégzett tesztek száma       

In [25]:
insert_table = df[['Dátum',
                   'Új esetek száma',
                   'Az új elhunytak száma naponta',
                   'Új mintavételek száma',
                   'Új gyógyultak naponta',
                   'Új beoltottak száma Magyarországon'
                  ]]
insert_table.columns = [['date',
                         'new_cases',
                         'new_deaths',
                         'new_samples',
                         'new_recoveries',
                         'new_vaccinated'
                        ]]
insert_table.fillna(0,inplace=True)
#insert_table['date'] = insert_table['date'].astype(str)
insert_table['new_cases'] = insert_table['new_cases'].astype(int)
insert_table['new_deaths'] = insert_table['new_deaths'].astype(int)
insert_table['new_samples'] = insert_table['new_samples'].astype(int)
insert_table['new_recoveries'] = insert_table['new_recoveries'].astype(int)
insert_table['new_vaccinated'] = insert_table['new_vaccinated'].astype(int)

,date,new_cases,new_deaths,new_samples,new_recoveries,new_vaccinated
0,2020-03-04,2,0,0,0,0
1,2020-03-05,2,0,0,0,0
2,2020-03-06,0,0,39,0,0
3,2020-03-07,1,0,50,0,0
4,2020-03-08,2,0,43,0,0
...,...,...,...,...,...,...
1025,2022-12-24,0,0,0,0,0
1026,2022-12-25,0,0,0,0,0
1027,2022-12-26,0,0,0,0,0
1028,2022-12-27,0,0,0,0,0


In [38]:
db_name = 'covid_19_db.sqlite'
con = sqlite3.connect(in_dir/db_name)
cur = con.cursor()

try:
    cur.execute("""CREATE TABLE covid_19_hun
                (id integer PRIMARY KEY, 
                date TEXT,
                new_cases INTEGER,
                new_deaths INTEGER,
                new_samples INTEGER,
                new_recoveries INTEGER,
                new_vaccinated INTEGER
                );
                """
               )
    for row in insert_table.iterrows():
        cur.execute(f"""INSERT INTO covid_19_hun
        VALUES (
        {row[0]},
        "{row[1][0].strftime('%Y-%m-%d')}",
        {int(row[1][1])},
        {int(row[1][2])},
        {int(row[1][3])},
        {int(row[1][4])},
        {int(row[1][5])}
        )
        """)
except (sqlite3.OperationalError, sqlite3.IntegrityError) as err:
    print(err)
    print("rolling back table creation..")
    con.rollback()
    
con.commit()
con.close()

creating covid_19_hun table..
inserting rows..
done
